In [1]:
from tensorflow.keras.optimizers import SGD, Adam
import numpy as np
from sklearn.manifold import TSNE
import os
from time import time
import Nmetrics
import matplotlib.pyplot as plt

from scipy.optimize import linear_sum_assignment as linear_assignment

from sklearn.neighbors import kneighbors_graph

import pydot
import pydotplus
import tensorflow as tf
from pydotplus import graphviz
from tensorflow.keras.utils import plot_model, model_to_dot

from time import time
import platform
from sklearn.metrics import log_loss
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Conv1D, Conv2D, Conv2DTranspose, Flatten, Reshape, Conv3D, Conv3DTranspose, MaxPooling2D, Dropout, GlobalMaxPooling2D
from tensorflow.keras.layers import Layer, InputSpec, Input, Dense, Multiply, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras import callbacks
from tensorflow.keras.initializers import VarianceScaling
from tensorflow.keras.regularizers import Regularizer, l1, l2, l1_l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering, SpectralClustering
from sklearn.decomposition import PCA, SparsePCA
from math import log

from keras_preprocessing import image
from PIL import Image
from numpy import hstack
from scipy import misc
import matplotlib
import scipy.io as scio
from sklearn.preprocessing import normalize
import warnings
warnings.filterwarnings("ignore")

path_data = './data'

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
np.random.seed(10)

2023-10-26 11:13:15.772094: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-26 11:13:15.806397: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-26 11:13:15.807096: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-26 11:13:16.464865: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Load Dataset

### Fashion_MV Dataset

In [2]:
def Fashion_MV():
    a = 0
    if a == 1:
        from tensorflow.keras.datasets import fashion_mnist  # this requires keras>=2.0.9
        (x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
        x1 = x_test
        y = y_test
        x2 = np.copy(x1)
        x3 = np.copy(x1)
        x4 = np.copy(x1)
        for i in range(len(y)):
            xb = np.where(y_train == y[i])
            xb = xb[0][0:2000]
            rand = np.random.randint(0, len(xb), 1)
            x2[i] = x_train[xb[rand]]
        for i in range(len(y)):
            xb = np.where(y_train == y[i])
            xb = xb[0][2000:4000]
            rand = np.random.randint(0, len(xb), 1)
            x3[i] = x_train[xb[rand]]
        for i in range(len(y)):
            xb = np.where(y_train == y[i])
            xb = xb[0][4000:6000]
            rand = np.random.randint(0, len(xb), 1)
            x4[i] = x_train[xb[rand]]
        # x1 = x1.reshape([-1, 28, 28, 1]) / 255.0
        x1 = x4.reshape([-1, 28, 28, 1]) / 255.0
        x2 = x2.reshape([-1, 28, 28, 1]) / 255.0
        x3 = x3.reshape([-1, 28, 28, 1]) / 255.0
        # The similar way of MNIST-USPS to construct Fashion-MV
        scio.savemat(path_data + '/3V_Fashion_MV.mat', {'X1': x1, 'X2': x2, 'X3': x3, 'Y': y})
    data = scio.loadmat(path_data + "/3V_Fashion_MV.mat")
    x1 = data['X1']
    x2 = data['X2']
    x3 = data['X3']
    Y = data['Y'][0]
    # ge = np.random.randint(0, len(x1), 1, dtype=int)
    # image1 = np.reshape(x1[ge], (28, 28))
    # image2 = np.reshape(x2[ge], (28, 28))
    # image3 = np.reshape(x3[ge], (28, 28))
    # print(Y[ge][0])
    # plt.figure('Fmnist-v1')
    # plt.imshow(image1)
    # plt.show()
    # plt.figure('Fmnist-v2')
    # plt.imshow(image2)
    # plt.show()
    # plt.figure('Fmnist-v3')
    # plt.imshow(image3)
    # plt.show()
    print(x1.shape)
    print(x2.shape)
    print(x3.shape)
    print(Y.shape)

    return [x1, x2, x3], Y


### MNIST-USPS Dataset

In [3]:
def Get_MNIST_USPS_From_COMIC():
    data = 0
    if data == 1:
        x = scio.loadmat(path_data + "/MNIST-USPS.mat")
        print(x)
        x1 = x['X1']
        x2 = x['X2']
        Y = x['Y']
        print(x1.shape)
        print(x2.shape)
        print(Y.shape)
        print(x1[0])
        print(x2[0])
        print(Y[0])
        x1 = x1.reshape((5000, 28, 28))
        x2 = x2.reshape((5000, 16, 16), order='A')
        print(Y)
        Y = Y[0].reshape(5000,)
        print(Y)
        xu_reshape = np.zeros([len(x2), 28, 28], dtype=float)
        for i in range(len(x2)):
            for x in range(16):
                for y in range(16):
                    xu_reshape[i][x + 6][y + 6] = x2[i][x][y]

        print(x1.shape)
        print(xu_reshape.shape)
        print(Y.shape)
        z = np.linspace(0, len(Y) - 1, len(Y), dtype=int)
        np.random.shuffle(z)
        # print(z)
        # print(y_label)
        x_data_m = x1
        x_data_u = xu_reshape
        y_label = Y
        x_shuffle_m = np.copy(x_data_m)
        x_shuffle_u = np.copy(x_data_u)
        y_shuffle = np.copy(y_label)
        for i in range(len(y_label)):
            x_shuffle_m[i] = x_data_m[z[i]]
            x_shuffle_u[i] = x_data_u[z[i]]
            y_shuffle[i] = y_label[z[i]]
        x_shuffle_m = x_shuffle_m.reshape([-1, 28, 28, 1])
        x_shuffle_u = x_shuffle_u.reshape([-1, 28, 28, 1])/255
        print(x_shuffle_m.shape)
        print(x_shuffle_u.shape)
        print(y_shuffle.shape)
        print(x_shuffle_m[0])
        print(x_shuffle_u[0])
        # print(y_shuffle[0])
        scio.savemat(path_data + '/2V_MNIST_USPS.mat', {'X1': x_shuffle_m, 'X2': x_shuffle_u, 'Y': y_shuffle})
    data = scio.loadmat(path_data + "/2V_MNIST_USPS.mat")
    x1 = data['X1']
    x2 = data['X2']
    Y = data['Y'][0]
    ge = np.random.randint(0, len(x1), 1, dtype=int)
    image1 = np.reshape(x1[ge], (28, 28))
    image2 = np.reshape(x2[ge], (28, 28))
#     print(Y[ge][0])
#     plt.figure('Mnist')
#     plt.imshow(image1)
#     plt.show()
#     plt.figure('USPS')
#     plt.imshow(image2)
#     plt.show()
    print(x1.shape)
    print(x2.shape)
    print(Y.shape)

    return [x1, x2], Y

### Caltech101_20 Dataset

In [4]:
def Caltech101_20():
    data = 0
    if data == 1:
        import scipy.io as scio
        data = scio.loadmat(path_data + "/Caltech101-20.mat")
        Y = data['Y'] - 1
        # print(Y.shape)
        X = data['X']
        print(X[0][0].shape)
        print(X[0][1].shape)
        print(X[0][2].shape)
        print(X[0][3].shape)
        print(X[0][4].shape)
        print(X[0][5].shape)
        x1 = X[0][0]
        x2 = X[0][1]
        x3 = X[0][2]
        x4 = X[0][3]
        x5 = X[0][4]
        x6 = X[0][5]
        t = np.linspace(0, Y.shape[0] - 1, Y.shape[0], dtype=int)
        print(t)
        import random
        random.shuffle(t)
        # np.save("./Caltech101_20_t.npy", t)
        t = np.load("./Caltech101_20_t.npy")
        print(t)
        xx1 = np.copy(x1)
        xx2 = np.copy(x2)
        xx3 = np.copy(x3)
        xx4 = np.copy(x4)
        xx5 = np.copy(x5)
        xx6 = np.copy(x6)
        YY = np.copy(Y)
        for i in range(Y.shape[0]):
            x1[i] = xx1[t[i]]
            x2[i] = xx2[t[i]]
            x3[i] = xx3[t[i]]
            x4[i] = xx4[t[i]]
            x5[i] = xx5[t[i]]
            x6[i] = xx6[t[i]]
            Y[i] = YY[t[i]]
        from sklearn import preprocessing
        min_max_scaler = preprocessing.MinMaxScaler()
        x1 = min_max_scaler.fit_transform(x1)
        x2 = min_max_scaler.fit_transform(x2)
        x3 = min_max_scaler.fit_transform(x3)
        x4 = min_max_scaler.fit_transform(x4)
        x5 = min_max_scaler.fit_transform(x5)
        x6 = min_max_scaler.fit_transform(x6)
        print(x1[0])
        Y = Y.reshape(Y.shape[0])
        print(Y.shape)
        scio.savemat(path_data + '/6V_Caltech101_20.mat', {'X1': x1, 'X2': x2, 'X3': x3, 'X4': x4, 'X5': x5, 'X6': x6, 'Y': Y})
    import scipy.io as scio
    data = scio.loadmat(path_data + "/6V_Caltech101_20.mat")
    x1 = data['X1']
    x2 = data['X2']
    x3 = data['X3']
    x4 = data['X4']
    x5 = data['X5']
    x6 = data['X6']
    Y = data['Y'][0]
    print(x1.shape)
    print(x2.shape)
    print(x3.shape)
    print(x4.shape)
    print(x5.shape)
    print(x6.shape)
    print(Y.shape)

    return [x1, x2, x3, x4, x5, x6], Y

### BDGP Dataset

In [5]:
def BDGP():
    data = scio.loadmat(path_data + "/2V_BDGP.mat")
    x1 = data['X1']
    x2 = data['X2']
    Y = data['Y'][0]
    print(x1.shape)
    print(x2.shape)
    print(Y.shape)
    return [x1, x2], Y

In [6]:
def load_data_conv(dataset):
    print("load:", dataset)
    if dataset == 'Fashion_MV':                   # Fashion-10K-3views
        return Fashion_MV()
    elif dataset == 'MNIST_USPS':                 # MNIST-USPS
        return Get_MNIST_USPS_From_COMIC()
    elif dataset == 'Caltech101_20':              # Caltech101_20
        return Caltech101_20()
    elif dataset == 'BDGP':                       # BDGP
        return BDGP()
    else:
        raise ValueError('Not defined for loading %s' % dataset)

## SDMVC File

### Fully connected Autoencoders

In [7]:
def FAE(dims, act='relu', view=1):
    
    
    n_stacks = len(dims) - 1
    
    init = VarianceScaling(scale=1. / 3., mode='fan_in', distribution='uniform')

    input_name = 'v'+str(view)+'_'
    
    # input
    x = Input(shape=(dims[0],), name='input' + str(view))

    h = x
    # internal layers in encoder
    for i in range(n_stacks-1):
        h = Dense(dims[i + 1], activation=act, kernel_initializer=init, name=input_name+'encoder_%d' % i)(h)
 
    # hidden layer
    h = Dense(100, kernel_initializer=init, name='embedding' + str(view))(h)  # hidden layer, features are extracted from here

    y = h
    # internal layers in decoder
    for i in range(n_stacks-1, 0, -1):
        y = Dense(dims[i], activation=act, kernel_initializer=init, name=input_name+'decoder_%d' % i)(y)

    # output
    y = Dense(dims[0], kernel_initializer=init, name=input_name+'decoder_0')(y)

    return Model(inputs=x, outputs=y, name=input_name+'Fae'), Model(inputs=x, outputs=h, name=input_name+'Fencoder')

### Multi-view Autoencoders

In [8]:
# def MAE(view=6, filters=[32, 64, 128, 10], view_shape = [1, 2, 3]):

def MAE(view=6, filters=[32, 64, 128, 100], view_shape = [1, 2, 3]):

    if len(view_shape[0]) == 1:
        typenet = 'f-f'          # Fully connected networks
    else:
        typenet = 'c-c'          # Convolution networks

    if typenet == 'c-c':
        input1_shape = view_shape[0]
        input2_shape = view_shape[1]

        if input1_shape[0] % 8 == 0:
            pad1 = 'same'
        else:
            pad1 = 'valid'
        print("----------------------")
        print(filters)
        input1 = Input(input1_shape, name='input1')
        x = Conv2D(filters[0], 5, strides=2, padding='same', activation='relu', name='conv1_v1')(input1)
        x = Conv2D(filters[1], 5, strides=2, padding='same', activation='relu', name='conv2_v1')(x)
        x = Conv2D(filters[2], 3, strides=2, padding=pad1, activation='relu', name='conv3_v1')(x)
        x = Flatten(name='Flatten1')(x)
        x1 = Dense(units=filters[3], name='embedding1')(x)
        x = Dense(units=filters[2]*int(input1_shape[0]/8)*int(input1_shape[0]/8), activation='relu',
                  name='Dense1')(x1)
        x = Reshape((int(input1_shape[0]/8), int(input1_shape[0]/8), filters[2]), name='Reshape1')(x)
        x = Conv2DTranspose(filters[1], 3, strides=2, padding=pad1, activation='relu', name='deconv3_v1')(x)
        x = Conv2DTranspose(filters[0], 5, strides=2, padding='same', activation='relu', name='deconv2_v1')(x)
        x = Conv2DTranspose(input1_shape[2], 5, strides=2, padding='same', name='deconv1_v1')(x)

        input2 = Input(input2_shape, name='input2')
        xn = Conv2D(filters[0], 5, strides=2, padding='same', activation='relu', name='conv1_v2')(input2)
        xn = Conv2D(filters[1], 5, strides=2, padding='same', activation='relu', name='conv2_v2')(xn)
        xn = Conv2D(filters[2], 3, strides=2, padding=pad1, activation='relu', name='conv3_v2')(xn)
        xn = Flatten(name='Flatten2')(xn)
        x2 = Dense(units=filters[3], name='embedding2')(xn)
        xn = Dense(units=filters[2] * int(input2_shape[0] / 8) * int(input2_shape[0] / 8), activation='relu',
                   name='Dense2')(x2)
        xn = Reshape((int(input2_shape[0] / 8), int(input2_shape[0] / 8), filters[2]), name='Reshape2')(xn)
        xn = Conv2DTranspose(filters[1], 3, strides=2, padding=pad1, activation='relu', name='deconv3_v2')(xn)
        xn = Conv2DTranspose(filters[0], 5, strides=2, padding='same', activation='relu', name='deconv2_v2')(xn)
        xn = Conv2DTranspose(input2_shape[2], 5, strides=2, padding='same', name='deconv1_v2')(xn)
        encoder1 = Model(inputs=input1, outputs=x1)
        encoder2 = Model(inputs=input2, outputs=x2)
        ae1 = Model(inputs=input1, outputs=x)
        ae2 = Model(inputs=input2, outputs=xn)

        if view == 2:
            return [ae1, ae2], [encoder1, encoder2]
        else:
            input3_shape = view_shape[2]
            input3 = Input(input3_shape, name='input3')
            xr = Conv2D(filters[0], 5, strides=2, padding='same', activation='relu', name='conv1_v3')(input3)
            xr = Conv2D(filters[1], 5, strides=2, padding='same', activation='relu', name='conv2_v3')(xr)
            xr = Conv2D(filters[2], 3, strides=2, padding=pad1, activation='relu', name='conv3_v3')(xr)
            xr = Flatten(name='Flatten3')(xr)
            x3 = Dense(units=filters[3], name='embedding3')(xr)
            xr = Dense(units=filters[2] * int(input3_shape[0] / 8) * int(input3_shape[0] / 8), activation='relu',
                       name='Dense3')(x3)
            xr = Reshape((int(input3_shape[0] / 8), int(input3_shape[0] / 8), filters[2]), name='Reshape3')(xr)
            xr = Conv2DTranspose(filters[1], 3, strides=2, padding=pad1, activation='relu', name='deconv3_v3')(xr)
            xr = Conv2DTranspose(filters[0], 5, strides=2, padding='same', activation='relu', name='deconv2_v3')(xr)
            xr = Conv2DTranspose(input2_shape[2], 5, strides=2, padding='same', name='deconv1_v3')(xr)

            encoder3 = Model(inputs=input3, outputs=x3)
            ae3 = Model(inputs=input3, outputs=xr)

            return [ae1, ae2, ae3], [encoder1, encoder2, encoder3]

    if typenet == 'f-f':
        ae = []
        encoder = []
        for v in range(view):
            ae_tmp, encoder_tmp = FAE(dims=[view_shape[v][0], 500, 500, 2000, 10], view=v + 1)
            ae.append(ae_tmp)
            encoder.append(encoder_tmp)
        
        return ae, encoder

### Clustering Layer

In [9]:
class ClusteringLayer(Layer):

    def __init__(self, n_clusters, weights=None, alpha=1.0, **kwargs):
        if 'input_shape' not in kwargs and 'input_dim' in kwargs:
            kwargs['input_shape'] = (kwargs.pop('input_dim'),)
        super(ClusteringLayer, self).__init__(**kwargs)
        self.n_clusters = n_clusters
        self.alpha = alpha
        self.initial_weights = weights
        self.input_spec = InputSpec(ndim=2)

    def build(self, input_shape):
        assert len(input_shape) == 2    
        input_dim = input_shape.as_list()[1]
        self.input_spec = InputSpec(dtype=K.floatx(), shape=(None, input_dim))
        self.clusters = self.add_weight(shape=(self.n_clusters, input_dim), initializer='glorot_uniform', name='clusters')
        if self.initial_weights is not None:
            self.set_weights(self.initial_weights)
            del self.initial_weights
        self.built = True

    def call(self, inputs, **kwargs):
        """ student t-distribution, as same as used in t-SNE algorithm.
                 q_ij = 1/(1+dist(x_i, u_j)^2), then normalize it.
        Arguments:
            inputs: the variable containing data, shape=(n_samples, n_features)
        Return:
            q: student's t-distribution, or soft labels for each sample. shape=(n_samples, n_clusters)
        """
        q = 1.0 / (1.0 + (K.sum(K.square(K.expand_dims(inputs, axis=1) - self.clusters), axis=2) / self.alpha))
        q **= (self.alpha + 1.0) / 2.0
        q = K.transpose(K.transpose(q) / K.sum(q, axis=1))
        return q

    def compute_output_shape(self, input_shape):
        assert input_shape and len(input_shape) == 2
        return input_shape[0], self.n_clusters

    def get_config(self):
        config = {'n_clusters': self.n_clusters}
        base_config = super(ClusteringLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

### Multi-view Discriminative Clustering

In [10]:
class MvDEC(object):
    def __init__(self, filters=[32, 64, 128, 10], view=6, n_clusters=20, alpha=1.0, view_shape = [1, 2, 3, 4, 5, 6]):
        np.random.seed(10)
        super(MvDEC, self).__init__()

        self.view_shape = view_shape
        self.filters = filters
        self.n_clusters = n_clusters
        self.alpha = alpha
        self.pretrained = False
        # prepare MvDEC model
        self.view = len(view_shape)


        self.AEs, self.encoders = MAE(view=self.view, filters=self.filters, view_shape=self.view_shape)
    
        Input = []
        Output = []
        Input_e = []
        Output_e = []
        clustering_layer = []

        for v in range(self.view):
            Input.append(self.AEs[v].input)
            Output.append(self.AEs[v].output)
            Input_e.append(self.encoders[v].input)
            Output_e.append(self.encoders[v].output)
            clustering_layer.append(ClusteringLayer(self.n_clusters, name='clustering'+str(v+1))(self.encoders[v].output))

        self.autoencoder = Model(inputs=Input, outputs=Output)    # xin _ xout

        self.encoder = Model(inputs=Input_e, outputs=Output_e)   # xin _ q

        Output_m = []
        for v in range(self.view):
            Output_m.append(clustering_layer[v])
            Output_m.append(Output[v])
        self.model = Model(inputs=Input, outputs=Output_m)   # xin _ q _ xout

    def pretrain(self, x, y, optimizer='adam', epochs=200, batch_size=256, save_dir='results/temp', verbose=0):

        print('Begin pretraining: ', '-' * 60)
        multi_loss = []
        for view in range(len(x)):
            multi_loss.append('mse')
        self.autoencoder.compile(optimizer=optimizer, loss=multi_loss)
        csv_logger = callbacks.CSVLogger(save_dir + '/T_pretrain_ae_log.csv')
        save = '/ae_weights.h5'
        cb = [csv_logger]
        
        if y is not None and verbose > 0:
            
            class PrintACC(callbacks.Callback):
                def __init__(self, x, y, flag=1):
                    self.x = x
                    self.y = y
                    self.flag = flag
                    super(PrintACC, self).__init__()

                def on_epoch_end(self, epoch, logs=None):
                    time = 1    #  show k-means results on z
                    if int(epochs / time) != 0 and (epoch+1) % int(epochs/time) != 0:
                        # print(epoch)
                        return
                    view_name = 'embedding' + str(self.flag)
                    feature_model = Model(self.model.input[self.flag - 1], self.model.get_layer(name=view_name).output)
                    
                    features = feature_model.predict(self.x)
                    km = KMeans(n_clusters=len(np.unique(self.y)), n_init=20, n_jobs=4)
                    y_pred = km.fit_predict(features)
                    
                    print('\n' + ' '*8 + '|==>  acc: %.4f,  nmi: %.4f  <==|'
                          % (Nmetrics.acc(self.y, y_pred), Nmetrics.nmi(self.y, y_pred)))

            for view in range(len(x)):
                cb.append(PrintACC(x[view], y, flag=view + 1))

        # begin pretraining
        t0 = time()
        self.autoencoder.fit(x, x, batch_size=batch_size, epochs=epochs, callbacks=cb, verbose=verbose)

        print('Pretraining time: ', time() - t0)
        self.autoencoder.save_weights(save_dir + save)
        print('Pretrained weights are saved to ' + save_dir + save)
        self.pretrained = True
        print('End pretraining: ', '-' * 60)

    def load_weights(self, weights):  # load weights of models
        self.model.load_weights(weights)

    def predict_label(self, x):  # predict cluster labels using the output of clustering layer
        input_dic = {}
        for view in range(len(x)):
            input_dic.update({'input' + str(view+1): x[view]})
        Q_and_X = self.model.predict(input_dic, verbose=0)
        y_pred = []
        for view in range(len(x)):
            # print(view)
            y_pred.append(Q_and_X[view*2].argmax(1))
        
        y_q = Q_and_X[(len(x)-1)*2]
        for view in range(len(x) - 1):
            y_q += Q_and_X[view*2]

        # y_q = y_q/len(x)
        y_mean_pred = y_q.argmax(1)
        return y_pred, y_mean_pred

    @staticmethod    
    def target_distribution(q):
        weight = q ** 2 / q.sum(0)
        # return q
        return (weight.T / weight.sum(1)).T

    def compile(self, optimizer='sgd', loss=['kld', 'mse'], loss_weights=[0.1, 1.0]):
        self.model.compile(optimizer=optimizer, loss=loss, loss_weights=loss_weights)

    def train_on_batch(self, xin, yout, sample_weight=None):
        return self.model.train_on_batch(xin, yout, sample_weight)

    # DEMVC
    def fit(self, arg, x, y, maxiter=2e4, batch_size=256, tol=1e-3, UpdateCoo=200, save_dir='./results/tmp'):
        
        print('Begin clustering:', '-' * 60)
        print('Update Coo:', UpdateCoo)
        save_interval = int(maxiter)  # only save the initial and final model
        print('Save interval', save_interval)
        
        # Step 1: initialize cluster centers using k-means
        t1 = time()
        ting = time() - t1
        print(ting)
        
        time_record = []
        time_record.append(int(ting))
        print(time_record)
        print('Initializing cluster centers with k-means.')
        kmeans = KMeans(n_clusters=self.n_clusters, n_init=100)
        
        input_dic = {}
        for view in range(len(x)):
            input_dic.update({'input' + str(view+1): x[view]})
        features = self.encoder.predict(input_dic)

        y_pred = []
        center = []

# --------------------------------------------
        
        c = 1
        if c == 1:
            for view in range(len(x)):
                y_pred.append(kmeans.fit_predict(features[view]))
                np.save('TC'+str(view+1)+'.npy', [kmeans.cluster_centers_])
                center.append(np.load('TC'+str(view+1)+'.npy'))
        else:
            from numpy import hstack
            from sklearn import preprocessing
            min_max_scaler = preprocessing.MinMaxScaler()
            # n_features = []
            # for view in range(len(x)):
            #     n_features.append(min_max_scaler.fit_transform(features[view]))
            #     # n_features.append(features[view])
            # z = hstack(n_features)
            z = hstack(features)
            print(features[0].shape, len(x), z.shape)
            y_pred.append(kmeans.fit_predict(z))
            for view in range(len(x)-1):
                y_pred.append(y_pred[0])
            print(kmeans.cluster_centers_.shape)
            centers = kmeans.cluster_centers_
            for view in range(len(x)):
                b = 10*view
                e = b + 10
                np.save('TC'+str(view+1)+'.npy', [centers[:, b:e]])
                center.append(np.load('TC'+str(view+1)+'.npy'))
# --------------------------------------------

        for view in range(len(x)):
            acc = np.round(Nmetrics.acc(y, y_pred[view]), 5)
            nmi = np.round(Nmetrics.nmi(y, y_pred[view]), 5)
            vmea = np.round(Nmetrics.vmeasure(y, y_pred[view]), 5)
            ari = np.round(Nmetrics.ari(y, y_pred[view]), 5)
            print('Start-'+str(view+1)+': acc=%.5f, nmi=%.5f, v-measure=%.5f, ari=%.5f' % (acc, nmi, vmea, ari))

        y_pred_last = []
        y_pred_sp = []
        for view in range(len(x)):
            y_pred_last.append(y_pred[view])
            y_pred_sp.append(y_pred[view])

        for view in range(len(x)):
            if arg.K12q == 0:
                self.model.get_layer(name='clustering'+str(view+1)).set_weights(center[view])
            else:
                self.model.get_layer(name='clustering'+str(view+1)).set_weights(center[arg.K12q - 1])

        # Step 2: deep clustering
        # logging file
        import csv
        import os
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        logfile = open(save_dir + '/log.csv', 'w')
        logwriter = csv.DictWriter(logfile, fieldnames=['iter', 'nmi', 'vmea', 'ari', 'loss'])
        logwriter.writeheader()

        index_array = np.arange(x[0].shape[0])
        index = 0
        
        Loss = []
        avg_loss = []
        for view in range(len(x)):
            Loss.append(0)
            avg_loss.append(0)

        flag = 1
        
        vf = arg.view_first

        update_interval = arg.UpdateCoo
        
        for ite in range(int(maxiter)):  # fine-turing

            if ite % update_interval == 0:

                Q_and_X = self.model.predict(input_dic)
                # Coo
                for view in range(len(x)):
                    y_pred_sp[view] = Q_and_X[view*2].argmax(1)

                # print(flag, (flag % len(x)))
                # view_num = len(x)
                q_index = (flag + vf - 1) % len(x)
                if q_index == 0:
                    q_index = len(x)
                p = self.target_distribution(Q_and_X[(q_index-1) * 2])  # q->p
                # print(q_index)
                flag += 1
                print('Next corresponding: p' + str(q_index))

                P = []
                if arg.Coo == 1:
                    for view in range(len(x)):
                        P.append(p)
                else:
                    for view in range(len(x)):
                        P.append(self.target_distribution(Q_and_X[view*2]))

                ge = np.random.randint(0, x[0].shape[0], 1, dtype=int)
                ge = int(ge)
                print('Number of sample:' + str(ge))
                for view in range(len(x)):
                    for i in Q_and_X[view*2][ge]:
                        print("%.3f  " % i, end="")
                    print("\n")

                # evaluate the clustering performance
                for view in range(len(x)):
                    avg_loss[view] = Loss[view] / update_interval
                
                for view in range(len(x)):
                    Loss[view] = 0.

                if y is not None:
                    for num in range(self.n_clusters):
                        same = np.where(y == num)   
                        same = np.array(same)[0]
                        Out = y_pred_sp[len(x)-1][same]
                        for view in range(len(x)-1):
                            Out += y_pred_sp[view][same]

                        out = Out
                        comp = y_pred_sp[0][same]
                        
                        for i in range(len(out)):
                            if Out[i]/len(x) == comp[i]:
                                out[i] = 0
                            else:
                                out[i] = 1
                        if (len(out) != 0):        # Simply calculate the scale of the alignment
                            print('%d, %.2f%%, %d' % (num,len(np.array(np.where(out == 0))[0]) * 100/len(out),len(same)))
                        else:
                            print('%d, %.2f%%. %d' % (num, 0, len(same)))

                    for view in range(len(x)):
                        acc = np.round(Nmetrics.acc(y, y_pred_sp[view]), 5)
                        nmi = np.round(Nmetrics.nmi(y, y_pred_sp[view]), 5)
                        vme = np.round(Nmetrics.vmeasure(y, y_pred_sp[view]), 5)
                        ari = np.round(Nmetrics.ari(y, y_pred_sp[view]), 5)
                        logdict = dict(iter=ite, nmi=nmi, vmea=vme, ari=ari, loss=avg_loss[view])
                        logwriter.writerow(logdict)
                        logfile.flush()
                        print('V'+str(view+1)+'-Iter %d: acc=%.5f, nmi=%.5f, v-measure=%.5f, ari=%.5f; loss=%.5f' % (
                        ite, acc, nmi, vme, ari, avg_loss[view]))

                    ting = time() - t1

            # train on batch
            idx = index_array[index * batch_size: min((index + 1) * batch_size, x[0].shape[0])]
            x_batch = []
            y_batch = []
            for view in range(len(x)):
                x_batch.append(x[view][idx])
                y_batch.append(P[view][idx])
                y_batch.append(x[view][idx])

            tmp = self.train_on_batch(xin=x_batch, yout=y_batch)   # [y, xn, y, x]
            
            for view in range(len(x)):
                Loss[view] += tmp[2*view+1]
            
            index = index + 1 if (index + 1) * batch_size <= x[0].shape[0] else 0
            # ite += 1

        # save the trained model
        logfile.close()
        print('saving model to:', save_dir + '/model_final.h5')
        self.model.save_weights(save_dir + '/model_final.h5')
        self.autoencoder.save_weights(save_dir + '/pre_model.h5')
        print('Clustering time: %ds' % (time() - t1))
        print('End clustering:', '-' * 60)
        
        Q_and_X = self.model.predict(input_dic)
        y_pred = []
        for view in range(len(x)):
            y_pred.append(Q_and_X[view*2].argmax(1))
        
        y_q = Q_and_X[(len(x)-1)*2]
        for view in range(len(x) - 1):
            y_q += Q_and_X[view*2]
        # y_q = y_q/len(x)
        y_mean_pred = y_q.argmax(1)
        return y_pred, y_mean_pred

    # SDMVC
    def new_fit(self, arg, x, y, maxiter=2e4, batch_size=256, tol=1e-3, UpdateCoo=200, save_dir='./results/tmp'):
        
        print('Begin clustering:', '-' * 60)
        print('Update Coo:', UpdateCoo)
        save_interval = int(maxiter)  # only save the initial and final model
        print('Save interval', save_interval)
        
        # Step 1: initialize cluster centers using k-means
        t1 = time()
        ting = time() - t1
        
        # print(ting)
        time_record = []
        time_record.append(int(ting))
        
        # print(time_record)
        print('Initializing cluster centers with k-means.')
        kmeans = KMeans(n_clusters=self.n_clusters, n_init=100)

        input_dic = {}
        for view in range(len(x)):
            input_dic.update({'input' + str(view + 1): x[view]})
        features = self.encoder.predict(input_dic)

        if len(x) <= 3:      # small trick: less view, more times to over arg.AR, so as to get high Aligned Rate
            arg.ARtime = 3
        else:
            arg.ARtime = 2

        y_pred = []
        center = []

        from numpy import hstack
        from sklearn import preprocessing
        min_max_scaler = preprocessing.MinMaxScaler()
# --------------------------------------------
        c = 1
        if c == 1:
            for view in range(len(x)):
                y_pred.append(kmeans.fit_predict(features[view]))
                # np.save('TC' + str(view + 1) + '.npy', [kmeans.cluster_centers_])
                # center.append(np.load('TC' + str(view + 1) + '.npy'))
                center.append([kmeans.cluster_centers_])

        elif c == 2:
            n_features = []
            for view in range(len(x)):
                # n_features.append(min_max_scaler.fit_transform(features[view]))
                n_features.append(features[view])
            z = hstack(n_features)
            print(features[0].shape, len(x), z.shape)
            y_pred.append(kmeans.fit_predict(z))
            for view in range(len(x) - 1):
                y_pred.append(y_pred[0])
            print(kmeans.cluster_centers_.shape)
            centers = kmeans.cluster_centers_
            # print(self.new_P(z, centers))
            new_P = self.new_P(z, centers)
            print(new_P.argmax(1))
            print(y_pred[0])
            for view in range(len(x)):
                b = 10 * view
                e = b + 10
                np.save('TC' + str(view + 1) + '.npy', [centers[:, b:e]])
                center.append(np.load('TC' + str(view + 1) + '.npy'))
        else:
            for view in range(len(x)):
                y_pred.append(kmeans.fit_predict(features[view]))
            print("random")
        # --------------------------------------------

        for view in range(len(x)):
            acc = np.round(Nmetrics.acc(y, y_pred[view]), 5)
            nmi = np.round(Nmetrics.nmi(y, y_pred[view]), 5)
            vmea = np.round(Nmetrics.vmeasure(y, y_pred[view]), 5)
            ari = np.round(Nmetrics.ari(y, y_pred[view]), 5)
            print('Start-' + str(view + 1) + ': acc=%.5f, nmi=%.5f, v-measure=%.5f, ari=%.5f' % (acc, nmi, vmea, ari))

        y_pred_last = []
        y_pred_sp = []
        for view in range(len(x)):
            y_pred_last.append(y_pred[view])
            y_pred_sp.append(y_pred[view])

        for view in range(len(x)):
            # break
            if arg.K12q == 0:
                self.model.get_layer(name='clustering' + str(view + 1)).set_weights(center[view])
            else:
                self.model.get_layer(name='clustering' + str(view + 1)).set_weights(center[arg.K12q - 1])

        # Step 2: deep clustering
        # logging file
        import csv
        import os
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        logfile = open(save_dir + '/log.csv', 'w')
        logwriter = csv.DictWriter(logfile, fieldnames=['iter', 'nmi', 'vmea', 'ari', 'loss'])
        logwriter.writeheader()

        index_array = np.arange(x[0].shape[0])
        index = 0

        Loss = []
        avg_loss = []
        kl_loss = []
        for view in range(len(x)):
            Loss.append(0)
            avg_loss.append(0)
            kl_loss.append(100000)

        update_interval = arg.UpdateCoo
        center_init = 0
        alignment = 0
        alignment_large = 0

        ACC = []
        NMI = []
        ARI = []
        vACC = []
        vNMI = []
        vARI = []
        Rate = []
        MVKLLoss = []
        ite = 0
        while True:
            if ite % update_interval == 0:
                print('\n')
                for view in range(len(x)):
                    avg_loss[view] = Loss[view] / update_interval
                    kl_loss[view] = kl_loss[view] / update_interval

                Q_and_X = self.model.predict(input_dic)
                # Coo
                for view in range(len(x)):
                    # print(Q_and_X[view * 2][0])
                    y_pred_sp[view] = Q_and_X[view * 2].argmax(1)

                features = self.encoder.predict(input_dic)
                mu = []
                for view in range(len(x)):
                    muu = self.model.get_layer(name='clustering' + str(view + 1)).get_weights()
                    # print(muu)
                    mu.append(muu)

                # np.save(save_dir + '/Features/' + str(ite) + '.npy', features)
                # np.save(save_dir + '/Mu/' + str(ite) + '.npy', mu)
                # print(features[0][0])
                # print(features[1][0])

                if len(x) >= 6:
                    scaler = 1
                else:
                    scaler = 0
                # print("scaler ? :"+str(scaler))
                # if views' number is too many (eg, >= 6), we can scale the features to [0,1] to build global features
                if scaler == 1:
                    n_features = []
                    for view in range(len(x)):
                        n_features.append(min_max_scaler.fit_transform(features[view]))
                    z = hstack(n_features)
                else:
                    z = hstack(features)

                kmean = KMeans(n_clusters=self.n_clusters, n_init=100)
                y_pred = kmean.fit_predict(z)     # k-means on global features
                # print(kmeans.cluster_centers_.shape)
                # print(y_pred[0:9])

                acc = np.round(Nmetrics.acc(y, y_pred), 5)
                nmi = np.round(Nmetrics.nmi(y, y_pred), 5)
                vmea = np.round(Nmetrics.vmeasure(y, y_pred), 5)
                ari = np.round(Nmetrics.ari(y, y_pred), 5)
                print('ACC=%.5f, NMI=%.5f, ARI=%.5f' % (acc, nmi, ari))
                ACC.append(acc)
                NMI.append(nmi)
                ARI.append(ari)
                the = np.sum(kl_loss) / len(x)
                # print(kl_loss)
                # print(Loss)
                # print(np.sum(kl_loss), np.sum(Loss))

                if y is not None:

                    scale = len(y)
                    for i in range(len(y)):
                        predict = y_pred_sp[0][i]
                        for view in range(len(x) - 1):
                            if predict == y_pred_sp[view + 1][i]:
                                continue
                            else:
                                scale -= 1
                                break

                    # alignment_before = alignment
                    alignment = (scale / len(y))
                    print('Aligned Ratio: %.2f%%. %d' % (alignment * 100, len(y)))
                    Rate.append(alignment)
                    tmpACC = []
                    tmpNMI = []
                    tmpARI = []
                    for view in range(len(x)):
                        acc = np.round(Nmetrics.acc(y, y_pred_sp[view]), 5)
                        nmi = np.round(Nmetrics.nmi(y, y_pred_sp[view]), 5)
                        vme = np.round(Nmetrics.vmeasure(y, y_pred_sp[view]), 5)
                        ari = np.round(Nmetrics.ari(y, y_pred_sp[view]), 5)
                        logdict = dict(iter=ite, nmi=nmi, vmea=vme, ari=ari, loss=avg_loss[view])
                        logwriter.writerow(logdict)
                        logfile.flush()
                        print('V' + str(view + 1) + '-Iter %d: ACC=%.5f, NMI=%.5f, ARI=%.5f; Loss=%.5f' % (ite, acc, nmi, ari, avg_loss[view]))
                        
                        tmpACC.append(acc)
                        tmpNMI.append(nmi)
                        tmpARI.append(ari)
                    vACC.append(tmpACC)
                    vNMI.append(tmpNMI)
                    vARI.append(tmpARI)
                    ting = time() - t1

                if alignment > arg.AR:
                    alignment_large += 1
                else:
                    alignment_large = 0
                # print("Over AR times:" + str(alignment_large))
                if alignment_large < arg.ARtime:
                    Center_init = kmean.cluster_centers_    # k-means on global features
                    new_P = self.new_P(z, Center_init)      # similarity measure
                    print("Self-Supervised Multi-View Discriminative Feature Learning")
                    p = self.target_distribution(new_P)     # enhance discrimination
                    center_init += 1
                else:
                    break
                P = []
                if arg.Coo == 1:
                    print("Unified Target Distribution for Multiple KL Losses")
                    for view in range(len(x)):
                        P.append(p)
                else:
                    print("self clustering")
                    for view in range(len(x)):
                        P.append(self.target_distribution(Q_and_X[view * 2]))

                # ge = np.random.randint(0, x[0].shape[0], 1, dtype=int)
                # ge = int(ge)
                # print('Number of sample:' + str(ge))
                # for view in range(len(x)):
                #     for i in Q_and_X[view * 2][ge]:
                #         print("%.3f  " % i, end="")
                #     print("\n")

                # evaluate the clustering performance
                for view in range(len(x)):
                    Loss[view] = 0.
                    kl_loss[view] = 0.

            # train on batch
            idx = index_array[index * batch_size: min((index + 1) * batch_size, x[0].shape[0])]
            x_batch = []
            y_batch = []
            for view in range(len(x)):
                x_batch.append(x[view][idx])
                y_batch.append(P[view][idx])
                y_batch.append(x[view][idx])

            tmp = self.train_on_batch(xin=x_batch, yout=y_batch)  # [sum, q, xn, q, x]
            # print(tmp)
            KLLoss = []
            for view in range(len(x)):
                Loss[view] += tmp[2 * view + 2]       # lr
                kl_loss[view] += tmp[2 * view + 1]    # lc
                # KLLoss += tmp[2 * view + 1]
                KLLoss.append(tmp[2 * view + 1])
            MVKLLoss.append(KLLoss)
            index = index + 1 if (index + 1) * batch_size <= x[0].shape[0] else 0
            # print(ite)
            ite += 1
            if ite >= int(maxiter):
                ite = 0
                # # Train from scratch
                # print("Pretrain self.autoencoder")
                # optimizer = Adam(lr=0.001)
                # self.pretrain(x, y, optimizer=optimizer, epochs=500, batch_size=batch_size,
                #               save_dir=save_dir, verbose=1)
                print("self.autoencoder.load_weights(args.pretrain_dir)")
                self.autoencoder.load_weights(arg.pretrain_dir)
                features = self.encoder.predict(input_dic)
                for view in range(len(x)):
                    kmeans.fit_predict(features[view])
                    self.model.get_layer(name='clustering' + str(view + 1)).set_weights([kmeans.cluster_centers_])

        # save the trained model
        logfile.close()
        print('saving model to:', save_dir + '/model_final.h5')
        self.model.save_weights(save_dir + '/model_final.h5')
        self.autoencoder.save_weights(save_dir + '/pre_model.h5')
        # np.save(save_dir + '/AccNmiAriRate/ACC.npy', ACC)
        # np.save(save_dir + '/AccNmiAriRate/NMI.npy', NMI)
        # np.save(save_dir + '/AccNmiAriRate/ARI.npy', ARI)
        # np.save(save_dir + '/AccNmiAriRate/vACC.npy', vACC)
        # np.save(save_dir + '/AccNmiAriRate/vNMI.npy', vNMI)
        # np.save(save_dir + '/AccNmiAriRate/vARI.npy', vARI)
        # np.save(save_dir + '/AccNmiAriRate/Rate.npy', Rate)
        # np.save(save_dir + '/AccNmiAriRate/MVKLLoss.npy', MVKLLoss)
        print('Clustering time: %ds' % (time() - t1))
        print('End clustering:', '-' * 60)

        Q_and_X = self.model.predict(input_dic)
        y_pred = []
        for view in range(len(x)):
            y_pred.append(Q_and_X[view * 2].argmax(1))

        y_q = Q_and_X[(len(x) - 1) * 2]
        for view in range(len(x) - 1):
            y_q += Q_and_X[view * 2]
        # y_q = y_q/len(x)
        y_mean_pred = y_q.argmax(1)
        return y_pred, y_mean_pred

    def new_P(self, inputs, centers):
        alpha = 1
        q = 1.0 / (1.0 + (np.sum(np.square(np.expand_dims(inputs, axis=1) - centers), axis=2) / alpha))
        q **= (alpha + 1.0) / 2.0
        q = np.transpose(np.transpose(q) / np.sum(q, axis=1))
        return q

### Make Data and Model

In [11]:
KNN_loss = []
diversity_loss = []

def _make_data_and_model(args):
    # prepare dataset
    x, y = load_data_conv(args.dataset)
    view = len(x)
    view_shapes = []
         
    Loss = []
    Loss_weights = []
    
    for v in range(view):
        view_shapes.append(x[v].shape[1:])
        Loss_weights.append(args.lc) # -----Kld
        Loss_weights.append(args.lm1) #-----Knn, Diversity, Elastic ------
    
    # prepare optimizer
    optimizer = Adam(lr=args.lr)
    # prepare the model
    n_clusters = len(np.unique(y))
    # n_clusters = 40   # over clustering
    print("n_clusters:" + str(n_clusters))
    # lc = 0.1
    
    print('K of Knn: ', k)
    print('Loss Weight: ', args.lm1)

    model = MvDEC(filters=[32, 64, 128, 10], n_clusters=n_clusters, view_shape=view_shapes)

    # get feature space from autoencoders
    
    encoder = model.encoders
    feature_space_matrix = []
    
    lam_Knn = 0.01
    lam_Div = 0.0001
    
    print('Lambda Knn: ', lam_Knn, ',  Lambda Div: ', lam_Div)

    # create feature matrix of encoder for each view

    if data == 'MNIST_USPS' or data == 'BDGP':  #2 views
        feature_space_matrix.append(encoder[0].predict(x[0]))
        feature_space_matrix.append(encoder[1].predict(x[1]))
    
    elif data == 'Fashion_MV':  #3 views
        feature_space_matrix.append(encoder[0].predict(x[0]))
        feature_space_matrix.append(encoder[1].predict(x[1]))
        feature_space_matrix.append(encoder[2].predict(x[2]))
        
    elif data == 'Caltech101_20':  #6 views
        feature_space_matrix.append(encoder[0].predict(x[0]))
        feature_space_matrix.append(encoder[1].predict(x[1]))
        feature_space_matrix.append(encoder[2].predict(x[2]))
        feature_space_matrix.append(encoder[3].predict(x[3]))
        feature_space_matrix.append(encoder[4].predict(x[4]))
        feature_space_matrix.append(encoder[5].predict(x[5]))
         
    print("###############################")    
    print('encoder[0]:', feature_space_matrix[0].shape)
    print('encoder[1]:', feature_space_matrix[1].shape)
    print("###############################")
    
    # trace of dot multiplication between laplacian_matrix and feature_space_matrix

    if data == 'BDGP':   #2 views
        l_f_1 = np.dot(laplacian_matrix1,feature_space_matrix[0])
        l_f_2 = np.dot(laplacian_matrix2,feature_space_matrix[1])

        graph_KNN_loss1 = l_f_1.trace()
        KNN_loss.append(lam_Knn * graph_KNN_loss1)

        graph_KNN_loss2 = l_f_2.trace()
        KNN_loss.append(lam_Knn * graph_KNN_loss2)
    
    elif data == 'MNIST_USPS':  #2 views
        #reshape
        feature_space_matrix[0] = feature_space_matrix[0].reshape((feature_space_matrix[0].shape[0], -1))
        feature_space_matrix[1] = feature_space_matrix[1].reshape((feature_space_matrix[1].shape[0], -1))
        
        l_f_1 = np.dot(laplacian_matrix1,feature_space_matrix[0])
        l_f_2 = np.dot(laplacian_matrix2,feature_space_matrix[1])

        graph_KNN_loss1 = l_f_1.trace()
        KNN_loss.append(lam_Knn * graph_KNN_loss1)

        graph_KNN_loss2 = l_f_2.trace()
        KNN_loss.append(lam_Knn * graph_KNN_loss2)
    
    elif data == 'Fashion_MV':  #3 views
        #reshape
        feature_space_matrix[0] = feature_space_matrix[0].reshape((feature_space_matrix[0].shape[0], -1))
        feature_space_matrix[1] = feature_space_matrix[1].reshape((feature_space_matrix[1].shape[0], -1))
        feature_space_matrix[2] = feature_space_matrix[2].reshape((feature_space_matrix[2].shape[0], -1))
        
        l_f_1 = np.dot(laplacian_matrix1,feature_space_matrix[0])
        l_f_2 = np.dot(laplacian_matrix2,feature_space_matrix[1])
        l_f_3 = np.dot(laplacian_matrix3,feature_space_matrix[2])

        graph_KNN_loss1 = l_f_1.trace()
        KNN_loss.append(lam_Knn * graph_KNN_loss1)

        graph_KNN_loss2 = l_f_2.trace()
        KNN_loss.append(lam_Knn * graph_KNN_loss2)
        
        graph_KNN_loss3 = l_f_3.trace()
        KNN_loss.append(lam_Knn * graph_KNN_loss3)
        
    elif data == 'Caltech101_20':  #6 views
        l_f_1 = np.dot(laplacian_matrix1,feature_space_matrix[0])
        l_f_2 = np.dot(laplacian_matrix2,feature_space_matrix[1])
        l_f_3 = np.dot(laplacian_matrix3,feature_space_matrix[2])
        l_f_4 = np.dot(laplacian_matrix4,feature_space_matrix[3])
        l_f_5 = np.dot(laplacian_matrix5,feature_space_matrix[4])
        l_f_6 = np.dot(laplacian_matrix6,feature_space_matrix[5])

        graph_KNN_loss1 = l_f_1.trace()
        KNN_loss.append(lam_Knn * graph_KNN_loss1)

        graph_KNN_loss2 = l_f_2.trace()
        KNN_loss.append(lam_Knn * graph_KNN_loss2)
        
        graph_KNN_loss3 = l_f_3.trace()
        KNN_loss.append(lam_Knn * graph_KNN_loss3)
                
        graph_KNN_loss4 = l_f_4.trace()
        KNN_loss.append(lam_Knn * graph_KNN_loss4)

        graph_KNN_loss5 = l_f_5.trace()
        KNN_loss.append(lam_Knn * graph_KNN_loss5)
        
        graph_KNN_loss6 = l_f_6.trace()
        KNN_loss.append(lam_Knn * graph_KNN_loss6)
    
#     print("--------- KNN_loss ---------------")
    
#     for i in range(len(KNN_loss)):
#         print(KNN_loss[i])
    
#     print("----------------------------------")
    
    
# Loss Diversity
        
    for i in range(len(feature_space_matrix)):
        
        if data == 'MNIST_USPS':
            sum_v = np.zeros((5000, 5000))
        
        elif data == 'Fashion_MV':
            sum_v = np.zeros((10000, 10000))
        
        elif data == 'BDGP':
            sum_v = np.zeros((2500, 2500))
            
        elif data == 'Caltech101_20':
            sum_v = np.zeros((2386, 2386))
            
        for j in range(len(feature_space_matrix)):
            if i == j:
                continue
            else:
                arr = np.dot(feature_space_matrix[i], tf.transpose(feature_space_matrix[j]))
                sum_v = sum_v + arr

        sum_trace = sum_v.trace()
        diversity_loss.append(lam_Div * sum_trace)
            
#     print("............... loss_diversity ......................")
    
#     for i in range(len(diversity_loss)):
#         print(diversity_loss[i])
    
#     print("......................................................")

    
    model.compile(optimizer=optimizer, loss=total_loss, loss_weights=Loss_weights)
    
    return x, y, model

In [12]:
def train(args):
    # get data and mode
    x, y, model = _make_data_and_model(args)

    model.model.summary()
    
    # pretraining
    t0 = time()
    if not os.path.exists(args.save_dir):
        os.makedirs(args.save_dir)
    if args.pretrain_dir is not None and os.path.exists(args.pretrain_dir):  # load pretrained weights
        model.autoencoder.load_weights(args.pretrain_dir)
        # model.load_weights(args.pretrain_dir)
    else:  # train
        optimizer = Adam(lr=args.lr)
        model.pretrain(x, y, optimizer=optimizer, epochs=args.pretrain_epochs,
                            batch_size=args.batch_size, save_dir=args.save_dir, verbose=args.verbose)
        args.pretrain_dir = args.save_dir + '/ae_weights.h5'
    t1 = time()
    print("Time for pretraining: %ds" % (t1 - t0))

    # clustering
    # DEMVC, IDEC, DEC
    # y_pred, y_mean_pred = model.fit(arg=args, x=x, y=y, maxiter=args.maxiter,
    #                                            batch_size=args.batch_size, UpdateCoo=args.UpdateCoo,
    #                                            save_dir=args.save_dir)
    # SDMVC
    y_pred, y_mean_pred = model.new_fit(arg=args, x=x, y=y, maxiter=args.maxiter,
                                    batch_size=args.batch_size, UpdateCoo=args.UpdateCoo,
                                    save_dir=args.save_dir)
    if y is not None:
        for view in range(len(x)):
            print('Final: acc=%.4f, nmi=%.4f, ari=%.4f' %
                    (Nmetrics.acc(y, y_pred[view]), Nmetrics.nmi(y, y_pred[view]), Nmetrics.ari(y, y_pred[view])))
        print('Final: acc=%.4f, nmi=%.4f, ari=%.4f' %
                  (Nmetrics.acc(y, y_mean_pred), Nmetrics.nmi(y, y_mean_pred), Nmetrics.ari(y, y_mean_pred)))

    t2 = time()
    print("Time for pretaining, clustering and total: (%ds, %ds, %ds)" % (t1 - t0, t2 - t1, t2 - t0))
    print('='*60)

In [13]:
def test(args):
    assert args.weights is not None

    x, y, model = _make_data_and_model(args)
    model.model.summary()
    print('Begin testing:', '-' * 60)
    model.load_weights(args.weights)
    y_pred, y_mean_pred = model.predict_label(x=x)
    if y is not None:
        for view in range(len(x)):
            print('Final: acc=%.4f, nmi=%.4f, ari=%.4f' %
                    (Nmetrics.acc(y, y_pred[view]), Nmetrics.nmi(y, y_pred[view]), Nmetrics.ari(y, y_pred[view])))
        print('Final: acc=%.4f, nmi=%.4f, ari=%.4f' %
                  (Nmetrics.acc(y, y_mean_pred), Nmetrics.nmi(y, y_mean_pred), Nmetrics.ari(y, y_mean_pred)))
    
    print('End testing:', '-' * 60)

## KNN

In [14]:
# data = 'BDGP'
data = 'MNIST_USPS'
# data = 'Fashion_MV'
# data = 'Caltech101_20'

x, y = load_data_conv(data)
k = 3
#reshape for mnist and fashion dataset
if data == 'MNIST_USPS':
    x[0] = x[0].reshape((x[0].shape[0], -1))
    x[1] = x[1].reshape((x[1].shape[0], -1))
    knn_m1 = kneighbors_graph(x[0], k, metric='cosine', mode='distance', include_self=True)
    knn_m2 = kneighbors_graph(x[1], k, metric='cosine', mode='distance', include_self=True)

elif data == 'Fashion_MV':
    x[0] = x[0].reshape((x[0].shape[0], -1))
    x[1] = x[1].reshape((x[1].shape[0], -1))
    x[2] = x[2].reshape((x[2].shape[0], -1))
    knn_m1 = kneighbors_graph(x[0], k, metric='cosine', mode='distance', include_self=True)
    knn_m2 = kneighbors_graph(x[1], k, metric='cosine', mode='distance', include_self=True)
    knn_m3 = kneighbors_graph(x[2], k, metric='cosine', mode='distance', include_self=True)

elif data == 'BDGP':
    knn_m1 = kneighbors_graph(x[0], k, metric='cosine', mode='distance', include_self=True)
    knn_m2 = kneighbors_graph(x[1], k, metric='cosine', mode='distance', include_self=True)

elif data == 'Caltech101_20':
    knn_m1 = kneighbors_graph(x[0], k, metric='cosine', mode='distance', include_self=True)
    knn_m2 = kneighbors_graph(x[1], k, metric='cosine', mode='distance', include_self=True)
    knn_m3 = kneighbors_graph(x[2], k, metric='cosine', mode='distance', include_self=True)
    knn_m4 = kneighbors_graph(x[3], k, metric='cosine', mode='distance', include_self=True)
    knn_m5 = kneighbors_graph(x[4], k, metric='cosine', mode='distance', include_self=True)
    knn_m6 = kneighbors_graph(x[5], k, metric='cosine', mode='distance', include_self=True)

load: Caltech101_20
(2386, 48)
(2386, 40)
(2386, 254)
(2386, 1984)
(2386, 512)
(2386, 928)
(2386,)


## Degree Matrix

In [15]:
if data == 'MNIST_USPS' or data == 'BDGP':  #2 views
    degree1 = np.diag(np.sum(knn_m1.toarray(), axis=1))
    degree2 = np.diag(np.sum(knn_m2.toarray(), axis=1))

elif data == 'Fashion_MV':  #3 views
    degree1 = np.diag(np.sum(knn_m1.toarray(), axis=1))
    degree2 = np.diag(np.sum(knn_m2.toarray(), axis=1))
    degree3 = np.diag(np.sum(knn_m3.toarray(), axis=1))
    
elif data == 'Caltech101_20':  #6 views
    degree1 = np.diag(np.sum(knn_m1.toarray(), axis=1))
    degree2 = np.diag(np.sum(knn_m2.toarray(), axis=1))
    degree3 = np.diag(np.sum(knn_m3.toarray(), axis=1))
    degree4 = np.diag(np.sum(knn_m4.toarray(), axis=1))
    degree5 = np.diag(np.sum(knn_m5.toarray(), axis=1))
    degree6 = np.diag(np.sum(knn_m6.toarray(), axis=1))

## Laplacian Matrix

In [16]:
if data == 'MNIST_USPS' or data == 'BDGP':
    laplacian_matrix1 = degree1 - knn_m1
    laplacian_matrix2 = degree2 - knn_m2
    
elif data == 'Fashion_MV':
    laplacian_matrix1 = degree1 - knn_m1
    laplacian_matrix2 = degree2 - knn_m2
    laplacian_matrix3 = degree3 - knn_m3
    
elif data == 'Caltech101_20':
    laplacian_matrix1 = degree1 - knn_m1
    laplacian_matrix2 = degree2 - knn_m2
    laplacian_matrix3 = degree3 - knn_m3    
    laplacian_matrix4 = degree4 - knn_m4
    laplacian_matrix5 = degree5 - knn_m5
    laplacian_matrix6 = degree6 - knn_m6

## Loss Functions

### Mean Squared Error

In [17]:
def mse_loss(y_true, y_pred):

    # Calculation of Mean Squared Error (MSE)
    ml = tf.keras.losses.MeanSquaredError()
    M_Loss = ml(y_true, y_pred)
    
    return M_Loss

### Kullback-Leibler divergence

In [18]:
def kld_loss(y_true, y_pred):

    # Computes Kullback-Leibler divergence loss (Kld)    
    kl = tf.keras.losses.KLDivergence()
    K_Loss = kl(y_true, y_pred)

    return K_Loss

### Elastic loss

In [19]:
delta = 100
print("Delta: ", delta)

def elastic_loss(y_true, y_pred):

    # Computes Elastic loss (El)
    
    y_true = tf.cast(y_true, tf.float32)

    a = tf.square(tf.subtract(y_true, y_pred))
    b = (delta*a) / (delta+a)
    c = a / (a+delta)
    
    d = b + c
    el_Loss = tf.reduce_sum(d)

    return el_Loss

Delta:  100


### Total Loss

In [20]:
def total_loss(y_true, y_pred):
        
    totalloss = []
    
    for v in range(len(x)):
        totalloss.append(kld_loss(y_true, y_pred))
#         totalloss.append(mse_loss(y_true, y_pred))
        totalloss.append(elastic_loss(y_true, y_pred) + KNN_loss[v].item() + diversity_loss[v].item())
#         totalloss.append(KNN_loss[v].item())
#         totalloss.append(diversity_loss[v].item())

    return totalloss

## Main

In [21]:
import sys
sys.argv=['']
del sys

In [22]:
if __name__ == "__main__":
    # -------------------------------------------------------
    # Dataset settings
    # 'MNIST_USPS'     # 2 views, 10 clusters,  5000 examples
    # 'Fashion_MV'     # 3 views, 10 clusters, 10000 examples
    # 'BDGP'           # 2 views,  5 clusters,  2500 examples
    # 'Caltech101_20'  # 6 views, 20 clusters,  2386 examples
    # -------------------------------------------------------
    data = 'MNIST_USPS'
#     TEST = True        # Test the clustering performance of the trained models
    TEST = False
    Train_ae = True  # The stability of AE’s pre-training and K-means might be the bottleneck for AE/K-means based MVC
#     Train_ae = False   # The reported results are the average values after pre-training

    AR = 0.90          # Aligned Ratio, e.g., 90%
    Coo = 1            # Unified P
    View = 1           # View_first SetC for DEMVC
    # K123q = View     # DEMVC
    K123q = 0          # SDMVC
    if Coo == 0:
        K123q = 0      # K-means 1：k1 , 2：k2, 3：k3, 0: k-means, >view number: no settings centers

    epochs = 500       # 500 epochs for pre-training AEs
    Update_Coo = 1000  # Iterations to update self-supervised objective
    Maxiter = 30000    # Max iterations for DEC, IDEC or DEMVC, not for SDMVC
    Batch = 256        # Batch size
    lc = 0.1           # Clustering loss = 0.1 --------- Kullback-Leibler
    Idec = 1.0         # Dec 0.0 , Idec 1.0 --------  Reconstruction loss 1.0 -------- MSE or ELASTIC
    lrate = 0.001      # Learning rate = 0.001 ---- keras defult

    lm_loss = 0.1      # weight loss = 0.1
    
    import argparse
    parser = argparse.ArgumentParser(description='main')

    parser.add_argument('--dataset', default=data,
                        help="Dataset name to train on")
    PATH = './results/'
    path = PATH + data
    if Train_ae:
        load = None
    else:
        load = path + '/ae_weights.h5'
    if TEST:
        load_test = path + '/model_final.h5'
    else:
        load_test = None
                        
    parser.add_argument('-d', '--save-dir', default=path,
                        help="Dir to save the results")
    # Parameters for pretraining
    parser.add_argument('--pretrain_dir', default=load, type=str,
                        help="Pretrained weights of the autoencoder")
    parser.add_argument('--pretrain-epochs', default=epochs, type=int,   # 500
                        help="Number of epochs for pretraining")
    parser.add_argument('-v', '--verbose', default=1, type=int,
                        help="Verbose for pretraining")
    # Parameters for clustering
    parser.add_argument('--testing', default=TEST, type=bool,
                        help="Testing the clustering performance with provided weights")
    parser.add_argument('--weights', default=load_test, type=str,
                        help="Model weights, used for testing")
    # pretrain_optimizer = 'adam'   # adam, sgd
    # parser.add_argument('--optimizer', default=pretrain_optimizer, type=str,
    #                     help="Optimizer for clustering phase")
    parser.add_argument('--lr', default=lrate, type=float,
                        help="learning rate during clustering")
    parser.add_argument('--batch-size', default=Batch, type=int,   # 256
                        help="Batch size")
    parser.add_argument('--maxiter', default=Maxiter, type=int,    # 2e4
                        help="Maximum number of iterations")
    parser.add_argument('-uc', '--UpdateCoo', default=Update_Coo, type=int,   # 200 
                        help="Number of iterations to update the target distribution")
    parser.add_argument('--view_first', default=View, type=int,
                        help="view-first")
    parser.add_argument('--Coo', default=Coo, type=int,
                        help="Coo?")
    parser.add_argument('--K12q', default=K123q, type=int,
                        help="Kmeans")
    parser.add_argument('--Idec', default=Idec, type=float,
                        help="dec?")
    parser.add_argument('--lc', default=lc, type=float,
                        help="Idec?")
    parser.add_argument('--AR', default=AR, type=float,
                        help="aligned rate?")
    parser.add_argument('--ARtime', default=1, type=float,
                        help="over aligned rate times?")
    
    parser.add_argument('--lm1', default=lm_loss, type=float,
                        help="loss")
    
    args = parser.parse_args()
    print('+' * 30, ' Parameters ', '+' * 30)
    print(args)
    print('+' * 75)
    
    # testing
    if args.testing:
        test(args)
    else:
        train(args)

++++++++++++++++++++++++++++++  Parameters  ++++++++++++++++++++++++++++++
Namespace(dataset='Caltech101_20', save_dir='./results/Caltech101_20', pretrain_dir=None, pretrain_epochs=1, verbose=1, testing=False, weights=None, lr=0.001, batch_size=256, maxiter=30000, UpdateCoo=1000, view_first=1, Coo=1, K12q=0, Idec=1.0, lc=0.1, AR=0.9, ARtime=1, lm1=0.1)
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
load: Caltech101_20
(2386, 48)
(2386, 40)
(2386, 254)
(2386, 1984)
(2386, 512)
(2386, 928)
(2386,)


2023-10-26 11:13:20.501388: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


n_clusters:20
K of Knn:  3
Loss Weight:  0.1
Lambda Knn:  0.01 ,  Lambda Div:  0.0001
75/75 [==============================] - 0s 2ms/step
###############################
encoder[0]: (2386, 100)
encoder[1]: (2386, 100)
###############################
Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input1 (InputLayer)         [(None, 48)]                 0         []                            
                                                                                                  
 input2 (InputLayer)         [(None, 40)]                 0         []                            
                                                                                                  
 input3 (InputLayer)         [(None, 254)]                0         []                            
                                       

                                                                                                  
 v3_decoder_2 (Dense)        (None, 500)                  1000500   ['v3_decoder_3[0][0]']        
                                                                                                  
 v4_decoder_2 (Dense)        (None, 500)                  1000500   ['v4_decoder_3[0][0]']        
                                                                                                  
 v5_decoder_2 (Dense)        (None, 500)                  1000500   ['v5_decoder_3[0][0]']        
                                                                                                  
 v6_decoder_2 (Dense)        (None, 500)                  1000500   ['v6_decoder_3[0][0]']        
                                                                                                  
 v1_decoder_1 (Dense)        (None, 500)                  250500    ['v1_decoder_2[0][0]']        
          

Begin pretraining:  ------------------------------------------------------------
75/75 [==============================] - 0s 2ms/steposs: 0.7339 - v1_decoder_0_loss: 0.0359 - v2_decoder_0_loss: 0.2002 - v3_decoder_0_loss: 0.0665 - v4_decoder_0_loss: 0.2413 - v5_decoder_0_loss: 0.0834 - v6_decoder_0_loss: 

        |==>  acc: 0.1324,  nmi: 0.1030  <==|
75/75 [==============================] - 0s 2ms/step

        |==>  acc: 0.2234,  nmi: 0.2875  <==|
75/75 [==============================] - 0s 2ms/step

        |==>  acc: 0.1764,  nmi: 0.1940  <==|
75/75 [==============================] - 0s 2ms/step

        |==>  acc: 0.2163,  nmi: 0.3300  <==|
75/75 [==============================] - 0s 2ms/step

        |==>  acc: 0.1697,  nmi: 0.1781  <==|
75/75 [==============================] - 0s 2ms/step

        |==>  acc: 0.1945,  nmi: 0.2077  <==|
10/10 [==============================] - 12s 787ms/step - loss: 0.7339 - v1_decoder_0_loss: 0.0359 - v2_decoder_0_loss: 0.2002 - v3_decoder_0_loss

KeyboardInterrupt: 